In [ ]:
from pandas.core.frame import DataFrame
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import time
import os

# Importing Files

In [ ]:
project_links_df = pd.read_excel('project_links.xlsx')
project_links_df.head()

In [ ]:
# creating a new column for the code in the project link

project_links_df['Project Link Code'] = project_links_df['Project Link'].apply(lambda x: x.split("/")[-1])
project_links_df.head()

### Function to rename files

In [ ]:
project_links_df['Project Link Code'].iloc[0:10]

In [ ]:
search_url = "https://data.archwaypartnership.uga.edu/"

browser = webdriver.Chrome()

# Open browser to begin search
browser.get(search_url)

# Maximize the window
browser.maximize_window()

time.sleep(2)

In [ ]:
# entering the username
input_username = browser.find_element(By.ID,'edit-name')
input_username.send_keys("bs83243")

time.sleep(1)

# entering the password
input_password = browser.find_element(By.ID,'edit-pass')
input_password.send_keys("BS@15july1997")


time.sleep(1)

# clicking the login button
button_login = browser.find_element(By.ID,'edit-submit')
button_login.click()
time.sleep(2)

In [ ]:
def get_table(browser):
    table = browser.find_elements(By.TAG_NAME, 'table')
    # Deliverable table is the 4th table on the page
    if len(table) < 4:
        return None
    return table[3]

def get_rows(table):
    tbody = table.find_elements(By.TAG_NAME, 'tbody')[0]
    return tbody.find_elements(By.TAG_NAME, 'tr')

In [ ]:
def check_next_page(browser):
    nav = browser.find_elements(By.CLASS_NAME, 'pager')
    if len(nav) > 0:
        nav_list = nav[0].find_elements(By.TAG_NAME, 'li')
        for li in nav_list:
            a_tag = li.find_elements(By.TAG_NAME, 'a')
            if a_tag and 'Go to next page' in a_tag[0].get_attribute('title'):
                a_tag[0].click()
                time.sleep(2)
                return True
    return False

In [ ]:
def rename_files_in_row(row, deliverable_folder):
    data = row.find_elements(By.TAG_NAME, 'td')[0]
    a_tag = data.find_element(By.TAG_NAME, 'a')
    del_link = a_tag.get_attribute('href')

    old_file_name = del_link.split('/')[-1]
    new_file_name = data.text

    old_file_path = os.path.join(deliverable_folder, old_file_name)
    new_file_path = os.path.join(deliverable_folder, new_file_name)

    if os.path.isfile(old_file_path):
        # There are some files with duplicate names
        if os.path.isfile(new_file_path):
            new_file_path = f"{new_file_path}_1"
        os.rename(old_file_path, new_file_path)
        print(f'Renamed {old_file_name} to {new_file_name}')

In [ ]:
def process_deliverables(browser, deliverable_folder):
    next_page_exists = True
    while next_page_exists:
        next_page_exists = False
        table = get_table(browser)
        if table is None:
            # No deliverables found
            break
        rows = get_rows(table)
        for row in rows:
            rename_files_in_row(row, deliverable_folder)
        next_page_exists = check_next_page(browser)

In [ ]:
# Load the CSV file with your data
df = project_links_df

# Iterate over the rows in the dataframe
for index, row in df.iterrows():
    code = str(row['Project Link Code'])
    project_link = row['Project Link']
    print(f'Processing project link: {project_link}')
    browser.get(project_link)
    time.sleep(2)
    
    # deliverable_folder = f'../../Downloads/Test-Rename/{code}-deliverables'
    deliverable_folder = f'/Users/bs83243/Downloads/Test-Deliverables-Extracted/{code}-deliverables'
    process_deliverables(browser, deliverable_folder)